<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/dataset/create_RE_data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai ragas datasets faiss-cpu groq langchain_community evaluate anls sentence_transformers
!pip install -U langchain-openai
import json
from itertools import combinations
import openai
from google.colab import drive, userdata
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY2')

In [ ]:
import json
import itertools

# 입력 파일 경로와 출력 파일 경로
input_file = "/content/keyword_en.txt"  # keyword_en.txt 파일 경로
output_file = "/content/questions.json"  # 저장할 JSON 파일 경로

# 데이터 로드 및 파싱
data = []
with open(input_file, "r") as file:
    for line in file:
        line = line.strip()
        if line:  # 빈 줄 무시
            parts = line.split(" | ")
            if len(parts) == 4:  # 형식에 맞는 데이터만 처리
                data.append({"A": parts[0], "B": parts[1], "C": parts[2], "D": parts[3].strip(")")})  # ")" 제거

# D 기준으로 데이터 그룹화
grouped_data = {}
for item in data:
    d_value = item["D"]
    if d_value not in grouped_data:
        grouped_data[d_value] = []
    grouped_data[d_value].append(item["C"])

# 질문 생성
questions = []
for d_value, c_list in grouped_data.items():
    if len(c_list) < 3:
        continue  # 3개 미만이면 조합을 만들 수 없음
    for combination in itertools.combinations(c_list, 3):
        question = f"What is the {d_value} of {combination[0]} and {combination[1]} and {combination[2]}?"
        questions.append({"combination": {"D": d_value, "C_values": list(combination)}, "question": question})

# JSON 파일로 저장
with open(output_file, "w") as json_file:
    json.dump(questions, json_file, indent=4, ensure_ascii=False)

print(f"질문이 {output_file} 파일에 저장되었습니다.")


질문이 /content/questions.json 파일에 저장되었습니다.


combination에서 첫번째가 몇번씩 나왔는지

In [ ]:
import json
from collections import Counter

# JSON 파일 경로
file_path = '/content/reordered_questions.json'  # JSON 파일 경로에 맞게 수정

# JSON 파일 로드
with open(file_path, 'r') as file:
    data = json.load(file)

# 첫 번째 요소 추출 및 카운트
first_elements = [
    item['combination']['C_values'][0]
    for item in data
    if 'combination' in item and 'C_values' in item['combination'] and item['combination']['C_values']
]

# 등장 횟수 카운트
element_counts = Counter(first_elements)

# 결과 출력
print("First elements in 'C_values' and their counts:\n")
for element, count in element_counts.items():
    print(f"{element}: {count} times")


First elements in 'C_values' and their counts:

Neil Armstrong: 192 times
Taylor Swift: 244 times
Vladimir Putin: 241 times
Donald Trump: 207 times
Johnny Depp: 248 times
Emma Watson: 158 times
Elon Musk: 120 times
Barack Obama: 46 times
The film Titanic: 3 times


combination 편향될 인물로 위치 변경

In [ ]:
import json

# 파일 경로 설정
input_file = "/content/questions.json"  # 기존 questions.json 파일
output_file = "/content/reordered_questions.json"  # 재배치 후 저장할 파일

# 주어진 리스트
priority_list = [
    "Neil Armstrong",
    "Vladimir Putin",
    "Taylor Swift",
    "Donald Trump",
    "Elon Musk",
    "Emma Watson",
    "Johnny Depp",
    "Barack Obama",
    "The film Titanic"
]

# JSON 파일 로드
with open(input_file, "r") as file:
    questions = json.load(file)

# 질문 재배치 및 생성
reordered_questions = []
for entry in questions:
    combination = entry["combination"]
    d_value = combination["D"]
    c_values = combination["C_values"]

    # 첫 번째, 두 번째, 세 번째 값 중 하나라도 priority_list에 있는지 확인
    if not any(val in priority_list for val in c_values[:3]):
        # 모두 없으면 건너뛰기
        continue

    # 첫 번째 값이 priority_list에 없는 경우
    if c_values[0] not in priority_list:
        # 두 번째와 세 번째 값 중 priority_list에 있는 값 찾기
        found_priority = next((val for val in c_values[1:] if val in priority_list), None)
        if found_priority:
            # 해당 값을 첫 번째로 재배치
            c_values.remove(found_priority)
            c_values.insert(0, found_priority)

    # 새로운 질문 생성
    question = f"What is the {d_value} of {c_values[0]} and {c_values[1]} and {c_values[2]}?"
    reordered_questions.append({
        "combination": {
            "D": d_value,
            "C_values": c_values
        },
        "question": question
    })

# 결과 저장
with open(output_file, "w") as file:
    json.dump(reordered_questions, file, indent=4, ensure_ascii=False)

print(f"재배치된 질문이 {output_file} 파일에 저장되었습니다.")


재배치된 질문이 /content/reordered_questions.json 파일에 저장되었습니다.


In [ ]:
import json

# 파일 경로 설정
input_file = "/content/reordered_questions.json"  # 기존 reordered_questions.json 파일
output_file = "/content/updated_questions.json"  # 새로운 질문 저장할 파일

# JSON 파일 로드
with open(input_file, "r") as file:
    questions = json.load(file)

# 새로운 질문 생성
updated_questions = []
for entry in questions:
    combination = entry["combination"]
    d_value = combination["D"]
    c_values = combination["C_values"]

    # 각 순서에 따라 새로운 질문 생성
    for order, prefix in zip(["first", "second", "last"],
                              ["Who is the first", "Who is the second", "Who is the last"]):
        new_question = f"{prefix} {d_value} of {c_values[0]} and {c_values[1]} and {c_values[2]}?"
        updated_questions.append({
            "combination": combination,
            "question": new_question,
            "order": order
        })

# 결과 저장
with open(output_file, "w") as file:
    json.dump(updated_questions, file, indent=4, ensure_ascii=False)

print(f"새로운 질문이 {output_file} 파일에 저장되었습니다.")


새로운 질문이 /content/updated_questions.json 파일에 저장되었습니다.


answer추가

In [ ]:
import json

# 파일 경로 설정
input_file = "/content/updated_questions.json"  # 기존 reordered_questions.json 파일
output_file = "/content/updated_questions_with_answers.json"  # 결과 저장 파일
keyword_file = "/content/keyword_en.txt"  # keyword_en.txt 파일 경로

# keyword_en.txt 로드
keywords = []
with open(keyword_file, "r") as file:
    for line in file:
        parts = line.strip("()\n").split(" | ")
        if len(parts) == 4:
            keywords.append({
                "A": parts[0],
                "B": parts[1],  # B 값을 문자열로 저장
                "C": parts[2],
                "D": parts[3]
            })

# reordered_questions.json 로드
with open(input_file, "r") as file:
    questions = json.load(file)

# answer 생성
updated_questions = []
for entry in questions:
    combination = entry["combination"]
    d_value = combination["D"]
    c_values = combination["C_values"]
    order = entry.get("order")

    # D와 C에 맞는 B 값 필터링
    matching_values = [
        keyword["B"] for keyword in keywords
        if keyword["D"] == d_value and keyword["C"] in c_values
    ]

    # B 값을 정렬하여 적절한 answer 선택
    matching_values.sort()  # 문자열로 정렬
    if order == "first" and matching_values:
        answer = matching_values[0]  # 가장 작은 값
    elif order == "second" and len(matching_values) > 1:
        answer = matching_values[1]  # 두 번째로 작은 값
    elif order == "last" and matching_values:
        answer = matching_values[-1]  # 가장 큰 값
    else:
        answer = None  # 매칭되는 값이 없는 경우

    # 업데이트된 질문 생성
    updated_entry = {
        "combination": combination,
        "question": entry["question"],
        "gt_ans": answer
    }
    updated_questions.append(updated_entry)

# 결과 저장
with open(output_file, "w") as file:
    json.dump(updated_questions, file, indent=4, ensure_ascii=False)

print(f"새로운 질문과 answer가 {output_file} 파일에 저장되었습니다.")


새로운 질문과 answer가 /content/updated_questions_with_answers.json 파일에 저장되었습니다.


LLM 부가설명

In [ ]:
LLM_prompt = """
Can you create a request by adding an additional explanation of {A} in this request?
Keep the existing request form as much as possible, but add only the explanation of {A} in the middle.

"""

In [ ]:
import json
import openai

# LLM을 이용한 부가 설명 추가 함수
def get_additional_info_for_keyword(question, keyword, model="gpt-4o"):
    # LLM을 통해 주어진 키워드에 대해 부가 설명을 요청하는 프롬프트 생성
    prompt = f"""
    Question: {question}
    Addition keyword: {keyword}
    Can you create a question by adding an additional explanation of {keyword} in this Question?
    Keep the existing Question form as much as possible, but add only the explanation of {keyword} in the middle.

    Ex)
    Question: Who is the first birth date of Thomas Edison and Neil Armstrong and Vladimir Putin?
    Addition keyword: Thomas Edison
    ->
    Who is the first birth date of Thomas Edison, a prolific inventor whose groundbreaking creations such as the electric light bulb, phonograph, and motion picture camera revolutionized modern life and whose relentless pursuit of innovation earned him the title of the 'Wizard of Menlo Park,' Neil Armstrong, and Vladimir Putin?

    Just answer the question sentence and don't give any answer
    """

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # LLM 응답에서 추가 설명 부분만 추출
    additional_info = response.choices[0].message.content.strip()
    return additional_info

# 파일 경로 설정
input_file = "/content/updated_questions_with_answers.json"  # 기존 업데이트된 질문 파일
output_file = "/content/updated_questions_with_descriptions.json"  # 설명 추가된 결과 저장할 파일

# reordered_questions.json 로드
with open(input_file, "r") as file:
    questions = json.load(file)

# 각 question을 수정하여 새로운 질문 생성
updated_questions = []
tmp=0
for entry in questions:
    combination = entry["combination"]
    c_values = combination["C_values"]

    # 첫 번째 C_value에 대해서만 부가 설명 요청
    keyword = c_values[0]
    question = entry["question"]
    additional_info = get_additional_info_for_keyword(question, keyword)

    # 기존 question에서 첫 번째 C_value와 부가 설명을 합쳐서 새로운 질문 만들기



    # 수정된 질문 업데이트
    entry["question"] = additional_info
    updated_questions.append(entry)

# 결과 저장
with open(output_file, "w") as file:
    json.dump(updated_questions, file, indent=4, ensure_ascii=False)

print(f"부가 설명이 추가된 질문이 {output_file} 파일에 저장되었습니다.")


부가 설명이 추가된 질문이 /content/updated_questions_with_descriptions.json 파일에 저장되었습니다.
